In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import json, time
from slugify import slugify
from pathlib import Path

In [2]:
class Issue():
    year = 0
    month = ''
    issue = ''
    pages = {}
    
    def __init__(self, year, month, issue, pages):
        self.year = year
        self.month = month
        self.issue = issue
        self.pages = pages
    
    def as_dict(self):
        return {
            'year': self.year,
            'month': self.month,
            'issue': self.issue,
            'pages': self.pages
        }
    
    def get_slug(self):
        return slugify(issue)
    
    def get_path(self):
        return Path(f'billboards/{self.year}/{self.month}/{self.get_slug()}.json')
    
    def save(self):
        data = self.as_dict()
        json_data = json.dumps(data)
        if not Path(self.get_path()).parent.exists():
            Path(self.get_path()).parent.mkdir(parents=True)
        Path(self.get_path()).write_text(json_data)
        
    def read(self):
        return json.loads(Path(self.get_path()).read_text())
    
    def exists(self):
        return self.get_path().exists()

In [3]:
try:
    b
except:
    b = webdriver.Firefox()

In [4]:
b.get('https://www-proquest-com.proxy.library.nyu.edu/eima/publication/686483?decadeSelected=1980%20-%201989&yearSelected=1933&monthSelected=06&issueNameSelected=01933Y06Y24%2423Jun%2B24%2C%2B1933%243b%2B%2BVol.%2B45%2B%242825%2429')

In [5]:
input('login please')

login please


''

In [6]:
b.get('https://www-proquest-com.proxy.library.nyu.edu/eima/publication/686483?decadeSelected=1980%20-%201989&yearSelected=1933&monthSelected=06&issueNameSelected=01933Y06Y24%2423Jun%2B24%2C%2B1933%243b%2B%2BVol.%2B45%2B%242825%2429')

In [9]:
for year in [1934, 1935, 1936, 1937, 1938, 1939, 1930, 1931, 1932, 1933, 1940]:
    year_select = Select(b.find_element_by_id('yearSelected'))
    year_elements = {int(x.text): x for x in year_select.options if int(x.text) >= 1930 and int(x.text) <= 1940}
    year_select.select_by_visible_text(year_elements[year].text)
    time.sleep(2)
    
    for month in ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']:
        month_select = Select(b.find_element_by_id('monthSelected'))
        month_options = [x for x in month_select.options]
        month_options.reverse()
        month_elements = {x.text: x for x in month_options}

        month_select.select_by_visible_text(month_elements[month].text)
        time.sleep(2)
        
        issue_select = Select(b.find_element_by_id('issueSelected'))
        issue_options = [x for x in issue_select.options]
        issue_options.reverse()
        issue_elements = {x.text: x for x in issue_options}
        
        year_url = b.current_url
        for issue in issue_elements:
            print()
            print(f'--> {month} {year} - {issue}')
            issue_data = Issue(year=year, month=month, issue=issue, pages={})
            
            if issue_data.exists():
                print(f'------> already exists')
                continue
            
            issue_select = Select(b.find_element_by_id('issueSelected'))
            issue_select.select_by_visible_text(issue)
            time.sleep(3)
            b.find_element_by_css_selector('input[value="View issue"]').click()
            time.sleep(5)
            
            try:
                b.find_element_by_css_selector('a[title="First page"]').click()
                time.sleep(5)
            except:
                time.sleep(3)
                try:
                    b.find_element_by_css_selector('a[title="First page"]').click()
                    time.sleep(5)
                except:
                    pass

            has_next = True

            page_info = {}

            while has_next:
                # get results
                all_results = b.find_elements_by_class_name('resultHeader')
                unique_pages = {x.find_element_by_class_name('titleAuthorETC').text: x.find_element_by_tag_name('a').get_attribute('href') for x in all_results}

                page_info = {**page_info, **unique_pages}

                try:
                    if b.find_element_by_css_selector('a[title="Next page"]'):
                        has_next = True
                    else:
                        has_next = False
                except:
                    has_next = False

                if has_next:
                    b.find_element_by_css_selector('a[title="Next page"]').click()
                    time.sleep(10)
                    
            issue_data.pages = page_info
            if issue_data.pages:
                issue_data.save()
                print(f'------> saved')
            else:
                print(f'------> error occurred, issue not saved')
            time.sleep(4)


--> January 1934 - Jan 6, 1934; Vol. 46 (1)
------> already exists

--> January 1934 - Jan 13, 1934; Vol. 46 (2)
------> already exists

--> January 1934 - Jan 20, 1934; Vol. 46 (3)
------> already exists

--> January 1934 - Jan 27, 1934; Vol. 46 (4)
------> already exists

--> February 1934 - Feb 10, 1934; Vol. 46 (6)
------> already exists

--> February 1934 - Feb 24, 1934; Vol. 46 (8)
------> saved

--> March 1934 - Mar 3, 1934; Vol. 46 (9)
------> already exists

--> March 1934 - Mar 10, 1934; Vol. 46 (10)
------> already exists

--> March 1934 - Mar 17, 1934; Vol. 46 (11)
------> already exists

--> April 1934 - Apr 7, 1934; Vol. 46 (14)
------> already exists

--> April 1934 - Apr 14, 1934; Vol. 46 (15)
------> already exists

--> April 1934 - Apr 21, 1934; Vol. 46 (16)
------> already exists

--> May 1934 - May 12, 1934; Vol. 46 (19)
------> already exists

--> June 1934 - Jun 16, 1934; Vol. 46 (24)
------> already exists

--> June 1934 - Jun 23, 1934; Vol. 46 (25)
------> alre

NoSuchElementException: Message: Could not locate element with visible text: Dec 21, 1935; Vol. 47 (51)
